<a href="https://colab.research.google.com/github/Michell203/mais-202/blob/main/MusicClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import json
import os
from glob import glob # allows us to list all files to a directory
import IPython
import IPython.display as ipd # to play the Audio Files

import librosa # main package for working with Audio Data
import librosa.display

In [ ]:
audio_files = glob("/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/*/*.wav") #list of all wav files in data set
samplerate = 22050
sampletrack = samplerate * 30
print("testing")

In [ ]:
print("jello world")
